how this notebook works

In [ ]:
# dependencies

import pandas as pd
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)

from ModelBuilderMethods import scaleColumns

setting up a dataset

splitting the dataset

balancing the dataset

normalization / scaling

categorical values one-hot encoding

selecting a model

print model performance metrics on test data